In [123]:
#Library Import

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import altair as alt
import scipy.stats as stats
from scipy.stats import norm

# import seaborn as sns
# from sklearn.neighbors import KernelDensity
# from bokeh.models import ColumnDataSource, Label, PrintfTickFormatter
# from bokeh.palettes import Dark2_5 as colors
# from bokeh.plotting import figure, show
# from bokeh.sampledata.cows import data as df
# from bokeh.io import output_notebook


In [124]:
#copied from main.ipynb
# #import pandas as pd


df_course = pd.read_csv('assets/student.course.csv')
df_record = pd.read_csv('assets/student.record.csv')
df_term = pd.read_table('assets/term.table.txt', delimiter="\t").fillna('Unknown')



df_term.columns = ['TERM_ID', 'TERM_DESCRIPTION']

terms = ['Fall', 'Winter', 'Summer', 'Spring', 'Unknown']
df_term['TERM_NAME'] = df_term['TERM_DESCRIPTION'].apply(lambda x: list(filter(lambda y: y!='', [t if t in x else '' for t in terms]))[0])
df_term['TERM_YEAR'] = df_term['TERM_DESCRIPTION'].apply(lambda x: x[-4:] if x[-4:].isnumeric() else None)

merge_course_df = pd.merge(
                    df_course,
                    df_term, 
                    how='left',
                    left_on='TERM',
                    right_on='TERM_ID')

terms_student = ['ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM']
merge_record_df = df_record.copy()
for term in terms_student:
    
    merge_record_df = pd.merge(
                        merge_record_df,
                        df_term, 
                        how='left',
                        left_on=term,
                        right_on='TERM_ID')
    
    

    merge_record_df.rename(columns={'TERM_ID': term + '_' + 'TERM_ID', 
                            'TERM_DESCRIPTION': term + '_' + 'TERM_DESCRIPTION', 
                            'TERM_NAME': term + '_' + 'TERM_NAME', 
                            'TERM_YEAR': term + '_' + 'TERM_YEAR'}, inplace=True)    
    
 


# df_term
# df_course
# df_record
# merge_course_df
merge_record_df

    
course_columns = ['ANONID', 'SUBJECT', 'CATALOG_NBR', 'GRD_PTS_PER_UNIT', 'GPAO', 'DIV',
                  'ANON_INSTR_ID', 'TERM', 'TERM_ID', 'TERM_DESCRIPTION', 'TERM_NAME', 'TERM_YEAR']

record_columns = ['MAJOR3_DESCR', 'MAJOR2_DESCR', 'MAJOR1_DESCR', 'HSGPA', 'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE',
                  'LAST_ACT_READ_SCORE', 'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE', 
                  'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE', 'SEX', 'STDNT_GROUP1', 'STDNT_GROUP2', 'MAJOR1_DEPT', 
                  'MAJOR2_DEPT', 'MAJOR3_DEPT', 'ANONID', 'ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM', 
                  'MAJOR3_TERM_TERM_ID', 'MAJOR3_TERM_TERM_DESCRIPTION', 'MAJOR3_TERM_TERM_NAME', 'MAJOR3_TERM_TERM_YEAR']   

# merge_course_df.describe()
# merge_record_df.describe()


# https://www.bls.gov/oes/tables.htm
# https://data.census.gov/table/ACSST1Y2015.S0101
# https://data.census.gov/table/ACSST1Y2015.S1501?t=Education&g=040XX00US06
# merge_record_df.sort_values(by=['MAJOR3_TERM_TERM_YEAR'], ascending=[True])['MAJOR3_TERM_TERM_YEAR'].unique()
# 1992 - 2015


# Occupational data by gender
# https://www.bls.gov/cps/cps_aa2015.htm

# https://www.axios.com/2020/06/30/half-us-population-unemployed
# the labor market is only at 50%?


merge_record_df[['ADMIT_TERM','ADMIT_TERM_TERM_ID','ADMIT_TERM_TERM_DESCRIPTION','ADMIT_TERM_TERM_NAME']]


,ADMIT_TERM,ADMIT_TERM_TERM_ID,ADMIT_TERM_TERM_DESCRIPTION,ADMIT_TERM_TERM_NAME
0,NaN,NaN,NaN,NaN
1,110.0,110.0,Fall 2008,Fall
2,63.0,63.0,Fall 1998,Fall
3,106.0,106.0,Fall 2007,Fall
4,83.0,83.0,Fall 2002,Fall
...,...,...,...,...
138883,NaN,NaN,NaN,NaN
138884,126.0,126.0,Fall 2012,Fall
138885,98.0,98.0,Fall 2005,Fall
138886,NaN,NaN,NaN,NaN


In [125]:
merge_record_df.columns

Index(['MAJOR3_DESCR', 'MAJOR2_DESCR', 'MAJOR1_DESCR', 'HSGPA',
       'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE', 'LAST_ACT_READ_SCORE',
       'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE',
       'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE', 'SEX', 'STDNT_GROUP1',
       'STDNT_GROUP2', 'MAJOR1_DEPT', 'MAJOR2_DEPT', 'MAJOR3_DEPT', 'ANONID',
       'ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM',
       'ADMIT_TERM_TERM_ID', 'ADMIT_TERM_TERM_DESCRIPTION',
       'ADMIT_TERM_TERM_NAME', 'ADMIT_TERM_TERM_YEAR', 'MAJOR1_TERM_TERM_ID',
       'MAJOR1_TERM_TERM_DESCRIPTION', 'MAJOR1_TERM_TERM_NAME',
       'MAJOR1_TERM_TERM_YEAR', 'MAJOR2_TERM_TERM_ID',
       'MAJOR2_TERM_TERM_DESCRIPTION', 'MAJOR2_TERM_TERM_NAME',
       'MAJOR2_TERM_TERM_YEAR', 'MAJOR3_TERM_TERM_ID',
       'MAJOR3_TERM_TERM_DESCRIPTION', 'MAJOR3_TERM_TERM_NAME',
       'MAJOR3_TERM_TERM_YEAR'],
      dtype='object')

In [126]:
table = pd.pivot_table(merge_record_df, values='ANONID', index=['ADMIT_TERM_TERM_YEAR','SEX'],
                       aggfunc="count")
table.reset_index(inplace = True)
#display(table)
area = alt.Chart(table).mark_area(opacity = 0.4).encode(
        x="ADMIT_TERM_TERM_YEAR",
        y=alt.Y('ANONID', stack = False),
        color="SEX",
        ).properties(title={"text": "number of students at university X per year grouped by sex"}
        )
area

alt.Chart(...)

In [127]:

result = merge_course_df.groupby('ANONID').agg(
    avg_GPAO=('GPAO', 'mean'),
    max_TERM_YEAR=('TERM_YEAR', 'max')
).reset_index()

result.head()
merge_df = pd.merge(
                    merge_record_df,
                    result, 
                    how='left',
                    left_on='ANONID',
                    right_on='ANONID')
#merge_df.head()

In [128]:
columnlst = ['HSGPA', 'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE', 'LAST_ACT_READ_SCORE',
       'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE',
       'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE','avg_GPAO']

def getFigs(timeCol, groupCol, columns, dataframe, agg='mean'):
    def layered_area_chart(col, data):
        area = alt.Chart(data).mark_area(opacity=0.4).encode(
            x=alt.X(f"{timeCol}:T", title="Year"),
            y=alt.Y(col, stack=False),
            color=alt.Color(groupCol)
        ).properties(
            title=f"{agg.capitalize()} {col} per year grouped by sex"
        )
        return area

    for col in columns:
        table = pd.pivot_table(dataframe, values=col, index=[timeCol, groupCol], aggfunc=agg)
        table.reset_index(inplace=True)
        
        # Generate and return chart (in Jupyter, returning should display the chart)
        fig = layered_area_chart(col, table)
        display(fig)  # Use display() in Jupyter environments

# Call the function to generate the charts
getFigs('ADMIT_TERM_TERM_YEAR', 'SEX', columnlst, merge_df)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [129]:
list1=[]
def getMajorType(x):
    
    if type(x) == str:
        lst = x.split(" ")
        deg = lst[-1]
        if lst[-1].startswith('B'):
            return lst[-1]
        elif lst[-1] == 'ABEd':
            return lst[-1]
        elif "BS" in lst:
            return 'BS'
        elif "BSE" in lst:
            return 'BSE'
        elif 'Nursing' in lst:
            return 'BSN'
        elif 'BFA' in lst:
            return 'BFA'
        else:
            list1.append(lst)
            return 'other'

merge_df["Deg_Type"] = merge_df['MAJOR1_DESCR'].apply(lambda x: getMajorType((x)))
merge_df['Deg_Type'].unique()
list2 = []
for x in list1:
    if x not in list2:
        list2.append(x)

for l in list2:
    print(' '.join(l))

len(list1)

Arab,Arm,Pers,Turk & Islamic S
Program in Entrepreneurship
Sustainability Scholars Prog
Earth System Science
Honors in the Liberal Arts
Unclassified


317

In [130]:
merge_df['Deg_Type'].unique()

array([None, 'BA', 'BSE', 'BS', 'BGS', 'BSChem', 'BBA', 'BFA', 'BMus',
       'other', 'BSEd', 'ABEd', 'BMusA', 'BSN', 'BDA', 'BTA', 'BFADes',
       'BioMed', 'BMU', 'BioE', 'BFM'], dtype=object)

In [131]:
getFigs('ADMIT_TERM_TERM_YEAR', 'Deg_Type', columnlst, merge_df)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [132]:
def getMajorType2(x):
    lst = [None, 'BA', 'BSE', 'BS', 'BGS', 'BSChem', 'BBA', 'BFA', 'BMus',
       'other', 'BSEd', 'ABEd', 'BMusA', 'BSN', 'BDA', 'BTA', 'BFADes',
       'BioMed', 'BMU', 'BioE', 'BFM']
    dic = {}
    dic['STEM'] = ['BSE', 'BS', 'BSChem', 'BSN', 'BTA', 'BioMed', 'BioE']
    dic['Arts&Humanities'] = ['BA', 'BGS', 'BFA', 'BMus', 'BMusA', 'BFADes','BMU']
    dic['Business'] = ['BBA', 'BFM']
    dic['Education'] = ['BSEd', 'ABEd']
    dic['other'] = ['other', 'BDA']

    if x in dic['STEM']:
        return 'STEM'
    elif x in dic['Arts&Humanities']:
        return 'Arts&Humanities'
    elif x in dic['Business']:
        return 'Business'
    elif x in dic['other']:
        return 'other'
    else:
        return
    
merge_df['Majot_Type'] = merge_df['Deg_Type']. apply(lambda x : getMajorType2(x))
merge_df.head()


,MAJOR3_DESCR,MAJOR2_DESCR,MAJOR1_DESCR,HSGPA,LAST_ACT_ENGL_SCORE,LAST_ACT_MATH_SCORE,LAST_ACT_READ_SCORE,LAST_ACT_SCIRE_SCORE,LAST_ACT_COMP_SCORE,LAST_SATI_VERB_SCORE,...,MAJOR2_TERM_TERM_NAME,MAJOR2_TERM_TERM_YEAR,MAJOR3_TERM_TERM_ID,MAJOR3_TERM_TERM_DESCRIPTION,MAJOR3_TERM_TERM_NAME,MAJOR3_TERM_TERM_YEAR,avg_GPAO,max_TERM_YEAR,Deg_Type,Majot_Type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.190000,2013,None,None
1,NaN,NaN,Asian Studies BA,3.8,25.0,22.0,29.0,28.0,26.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.477531,2010,BA,Arts&Humanities
2,NaN,NaN,Psychology BA,3.3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.971542,2002,BA,Arts&Humanities
3,NaN,NaN,Civil Engineering BSE,3.9,33.0,32.0,28.0,30.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.203586,2008,BSE,STEM
4,NaN,NaN,NaN,3.8,24.0,22.0,18.0,18.0,21.0,610.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.931080,2003,None,None


In [133]:
getFigs('ADMIT_TERM_TERM_YEAR', 'Majot_Type', columnlst, merge_df)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [134]:
major_df = pd.read_excel("assets/majors.xlsx")

In [135]:
merge_df.head()

,MAJOR3_DESCR,MAJOR2_DESCR,MAJOR1_DESCR,HSGPA,LAST_ACT_ENGL_SCORE,LAST_ACT_MATH_SCORE,LAST_ACT_READ_SCORE,LAST_ACT_SCIRE_SCORE,LAST_ACT_COMP_SCORE,LAST_SATI_VERB_SCORE,...,MAJOR2_TERM_TERM_NAME,MAJOR2_TERM_TERM_YEAR,MAJOR3_TERM_TERM_ID,MAJOR3_TERM_TERM_DESCRIPTION,MAJOR3_TERM_TERM_NAME,MAJOR3_TERM_TERM_YEAR,avg_GPAO,max_TERM_YEAR,Deg_Type,Majot_Type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.190000,2013,None,None
1,NaN,NaN,Asian Studies BA,3.8,25.0,22.0,29.0,28.0,26.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.477531,2010,BA,Arts&Humanities
2,NaN,NaN,Psychology BA,3.3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.971542,2002,BA,Arts&Humanities
3,NaN,NaN,Civil Engineering BSE,3.9,33.0,32.0,28.0,30.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.203586,2008,BSE,STEM
4,NaN,NaN,NaN,3.8,24.0,22.0,18.0,18.0,21.0,610.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.931080,2003,None,None


In [136]:
merge_df = pd.merge(
                    merge_df,
                    major_df, 
                    how='left',
                    left_on='MAJOR1_DESCR',
                    right_on='MAJOR')
merge_df.head()

,MAJOR3_DESCR,MAJOR2_DESCR,MAJOR1_DESCR,HSGPA,LAST_ACT_ENGL_SCORE,LAST_ACT_MATH_SCORE,LAST_ACT_READ_SCORE,LAST_ACT_SCIRE_SCORE,LAST_ACT_COMP_SCORE,LAST_SATI_VERB_SCORE,...,MAJOR3_TERM_TERM_ID,MAJOR3_TERM_TERM_DESCRIPTION,MAJOR3_TERM_TERM_NAME,MAJOR3_TERM_TERM_YEAR,avg_GPAO,max_TERM_YEAR,Deg_Type,Majot_Type,MAJOR,OCCUPATION
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.190000,2013,None,None,NaN,NaN
1,NaN,NaN,Asian Studies BA,3.8,25.0,22.0,29.0,28.0,26.0,NaN,...,NaN,NaN,NaN,NaN,3.477531,2010,BA,Arts&Humanities,Asian Studies BA,"Education, training, and library occupations"
2,NaN,NaN,Psychology BA,3.3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.971542,2002,BA,Arts&Humanities,Psychology BA,"Life, physical, and social science occupations"
3,NaN,NaN,Civil Engineering BSE,3.9,33.0,32.0,28.0,30.0,31.0,NaN,...,NaN,NaN,NaN,NaN,3.203586,2008,BSE,STEM,Civil Engineering BSE,Architecture and engineering occupations
4,NaN,NaN,NaN,3.8,24.0,22.0,18.0,18.0,21.0,610.0,...,NaN,NaN,NaN,NaN,1.931080,2003,None,None,NaN,NaN


In [137]:
getFigs('ADMIT_TERM_TERM_YEAR', 'OCCUPATION', columnlst, merge_df)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [186]:
#from main.ipynb

import pandas as pd

# More tables here
# https://www.bls.gov/cps/tables.htm
# BLS MEDAIN WEEKLY EARNINGS https://www.bls.gov/cps/cpsaat39.htm

# https://data.bls.gov/apps/demographic-comparison-matrix/home.htm

# treemaps https://docs.bokeh.org/en/latest/docs/examples/topics/hierarchical/treemap.html

# https://data.census.gov/table/ACSST1Y2015.S1501?t=Education&g=040XX00US06

df_bls_2015_headers_1 = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=0).fillna('Unknown').iloc[0]
df_bls_2015_headers_2 = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=1).fillna('Unknown').iloc[0]
df_bls_2015_headers_3 = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=2).fillna('Unknown').iloc[0]
df_bls_2015_headers_4 = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=3).fillna('Unknown').iloc[0]

df_bls_2015 = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=3).fillna('Unknown')

df_bls_2015

tabs = ['2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003']

columns = ['level', 
           'l3','l2','l1','l0',
           'occupation', 
           'total_16_years_and_over_py', 
           'total_16_years_and_over',
           'male_16_years_and_over_py', 
           'male_16_years_and_over',
           'male_20_years_and_over_py', 
           'male_20_years_and_over',
           'female_16_years_and_over_py', 
           'female_16_years_and_over',
           'female_20_years_and_over_py', 
           'female_20_years_and_over']

# combined = list(zip(df_bls_2015_headers_1, df_bls_2015_headers_2, df_bls_2015_headers_3, df_bls_2015_headers_4))
# for row in combined:
#     print(row)


df_bls_all = pd.DataFrame()
for tab in tabs:
    df_bls_next = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', 
                                sheet_name=str(tab), 
                                header=3, 
                                ).fillna('Unknown')
    
    df_bls_next.columns = columns
    df_bls_next['year'] = str(tab)
    # df_bls_all.columns = columns.append('year')
    df_bls_all = pd.concat([df_bls_all, df_bls_next])

#print(len(df_bls_all))
df_bls_all_filtered = df_bls_all[df_bls_all['occupation'] != 'Unknown']
df_bls_all_filtered_Totals = df_bls_all[(df_bls_all['level'] == 'L3') & (df_bls_all['total_16_years_and_over_py'] != 'Unknown')]
df_bls_all_filtered_L2 = df_bls_all[(df_bls_all['level'] == 'L2') & (df_bls_all['total_16_years_and_over_py'] != 'Unknown')]
df_bls_all_filtered_L0 = df_bls_all[(df_bls_all['level'] == 'L0') & (df_bls_all['total_16_years_and_over_py'] != 'Unknown')]


# Check L0 Sums vs Totals
# for tab in tabs:
#     print(tab, df_bls_all_filtered_L0[df_bls_all_filtered_L0['year'] == tab]['total_16_years_and_over_py'].sum(axis=0) - df_bls_all_filtered_Totals[df_bls_all_filtered_Totals['year'] == tab]['total_16_years_and_over_py'].sum(axis=0))


# print(len(df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == '2015')]))
# print(df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == '2015')].columns)
# print(df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == '2015')]['occupation'].unique())

# Sanity Check trend
#for tab in tabs:
 #   print(tab, df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == tab) & (df_bls_all_filtered_L0['occupation'] == 'Computer and mathematical occupations')]['total_16_years_and_over_py'].sum(axis=0))


df_bls_all_filtered


,level,l3,l2,l1,l0,occupation,total_16_years_and_over_py,total_16_years_and_over,male_16_years_and_over_py,male_16_years_and_over,male_20_years_and_over_py,male_20_years_and_over,female_16_years_and_over_py,female_16_years_and_over,female_20_years_and_over_py,female_20_years_and_over,year
0,L3,Unknown,Unknown,Unknown,Unknown,Total,146305.0,148834.0,77692.0,79131.0,75471.0,76776.0,68613.0,69703.0,66287.0,67323.0,2015
2,L2,Unknown,Unknown,Unknown,Unknown,"Management, professional, and related occupations",56050.0,57960.0,27119.0,28090.0,26976.0,27918.0,28931.0,29871.0,28751.0,29658.0,2015
3,L1,Unknown,Unknown,Unknown,Unknown,"Management, business, and financial operations...",23171.0,24108.0,13041.0,13589.0,13008.0,13548.0,10129.0,10519.0,10098.0,10476.0,2015
4,L0,Total,"Management, professional, and related occupations","Management, business, and financial operations...",Management occupations,Management occupations,16199.0,16994.0,9953.0,10340.0,9931.0,10309.0,6246.0,6654.0,6223.0,6624.0,2015
5,L0,Total,"Management, professional, and related occupations","Management, business, and financial operations...",Business and financial operations occupations,Business and financial operations occupations,6972.0,7114.0,3088.0,3249.0,3077.0,3238.0,3884.0,3866.0,3875.0,3851.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,L0,Total,"Natural resources, construction, and maintenan...","Natural resources, construction, and maintenan...",Office and administrative support occupations,Construction and extraction occupations,7898.0,8114.0,7674.0,7891.0,7431.0,7636.0,224.0,223.0,215.0,214.0,2003
41,L0,Total,"Natural resources, construction, and maintenan...","Natural resources, construction, and maintenan...",Office and administrative support occupations,"Installation, maintenance, and repair occupations",4623.0,5041.0,4412.0,4830.0,4312.0,4730.0,212.0,211.0,206.0,204.0,2003
44,L2,Unknown,Unknown,Unknown,Unknown,"Production, transportation, and material movin...",18569.0,18020.0,14091.0,13745.0,13509.0,13248.0,4478.0,4274.0,4334.0,4143.0,2003
45,L0,Total,"Natural resources, construction, and maintenan...","Natural resources, construction, and maintenan...",Office and administrative support occupations,Production occupations,10081.0,9700.0,6863.0,6696.0,6682.0,6566.0,3218.0,3004.0,3144.0,2938.0,2003


In [187]:
print(len(df_bls_all_filtered['occupation'].unique()))
print(len(df_bls_all_filtered['l0'].unique()))
print(len(df_bls_all_filtered['l1'].unique()))
print(len(df_bls_all_filtered['l2'].unique()))
df_bls_all_filtered.columns

31
17
6
5


Index(['level', 'l3', 'l2', 'l1', 'l0', 'occupation',
       'total_16_years_and_over_py', 'total_16_years_and_over',
       'male_16_years_and_over_py', 'male_16_years_and_over',
       'male_20_years_and_over_py', 'male_20_years_and_over',
       'female_16_years_and_over_py', 'female_16_years_and_over',
       'female_20_years_and_over_py', 'female_20_years_and_over', 'year'],
      dtype='object')

In [197]:
df_bls_all.reset_index(inplace= True)

In [258]:
def getMales(row):
    x = row['male_16_years_and_over'] +row['male_20_years_and_over']
    if type(x) ==float:
        return x
    else:
        return
df_bls_all_filtered['Males'] = df_bls_all_filtered.apply(lambda x: getMales(x), axis = 1)

def getFemales(row):
    x= row['female_16_years_and_over'] +row['female_20_years_and_over']
    if type(x) ==float:
            return x
    else:
        return
df_bls_all_filtered['Females'] = df_bls_all_filtered.apply(lambda x: getFemales(x), axis = 1)

df_bls_all_filtered.head()

/var/folders/0z/gp51c2ln7l595qj_bjp3g4y00000gn/T/ipykernel_62819/3496776192.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bls_all_filtered['Males'] = df_bls_all_filtered.apply(lambda x: getMales(x), axis = 1)
/var/folders/0z/gp51c2ln7l595qj_bjp3g4y00000gn/T/ipykernel_62819/3496776192.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bls_all_filtered['Females'] = df_bls_all_filtered.apply(lambda x: getFemales(x), axis = 1)


,level,l3,l2,l1,l0,occupation,total_16_years_and_over_py,total_16_years_and_over,male_16_years_and_over_py,male_16_years_and_over,male_20_years_and_over_py,male_20_years_and_over,female_16_years_and_over_py,female_16_years_and_over,female_20_years_and_over_py,female_20_years_and_over,year,Males,Females
0,L3,Unknown,Unknown,Unknown,Unknown,Total,146305.0,148834.0,77692.0,79131.0,75471.0,76776.0,68613.0,69703.0,66287.0,67323.0,2015,155907.0,137026.0
2,L2,Unknown,Unknown,Unknown,Unknown,"Management, professional, and related occupations",56050.0,57960.0,27119.0,28090.0,26976.0,27918.0,28931.0,29871.0,28751.0,29658.0,2015,56008.0,59529.0
3,L1,Unknown,Unknown,Unknown,Unknown,"Management, business, and financial operations...",23171.0,24108.0,13041.0,13589.0,13008.0,13548.0,10129.0,10519.0,10098.0,10476.0,2015,27137.0,20995.0
4,L0,Total,"Management, professional, and related occupations","Management, business, and financial operations...",Management occupations,Management occupations,16199.0,16994.0,9953.0,10340.0,9931.0,10309.0,6246.0,6654.0,6223.0,6624.0,2015,20649.0,13278.0
5,L0,Total,"Management, professional, and related occupations","Management, business, and financial operations...",Business and financial operations occupations,Business and financial operations occupations,6972.0,7114.0,3088.0,3249.0,3077.0,3238.0,3884.0,3866.0,3875.0,3851.0,2015,6487.0,7717.0


In [259]:
columnlst = ['Males', "Females"]
getFigs('year', 'l2', columnlst, df_bls_all_filtered, agg = 'sum')

alt.Chart(...)

alt.Chart(...)

In [260]:
table = pd.pivot_table(df_bls_all_filtered, values=['Males', 'Females'], index=['year', 'l2'],
                       aggfunc="sum")
table.reset_index(inplace = True)
table = table[table.l2 != 'Unknown']
display(table)
# Step 1: Use pd.melt to "unpivot" Females and Males columns
df_melted = pd.melt(table, 
                    id_vars=['year', 'l2'],  # Columns to keep
                    value_vars=['Females', 'Males'],  # Columns to unpivot
                    var_name='SEX',  # Name for new column indicating Sex (Females/Males)
                    value_name='count')  # Name for new column holding the values


# Step 3: Set 'year' and 'SEX' as the index
#df_melted.set_index(['year', 'SEX'], inplace=True)

# Display the transformed DataFrame
df_melted


,year,l2,Females,Males
0,2003,"Management, professional, and related occupations",48174.0,47332.0
1,2003,"Natural resources, construction, and maintenan...",9706.0,53638.0
2,2003,Sales and office occupations,43911.0,24907.0
3,2003,Service occupations,24020.0,17868.0
5,2004,"Management, professional, and related occupations",48596.0,48123.0
6,2004,"Natural resources, construction, and maintenan...",9405.0,54537.0
7,2004,Sales and office occupations,43929.0,24832.0
8,2004,Service occupations,24569.0,18602.0
10,2005,"Management, professional, and related occupations",49586.0,48551.0
11,2005,"Natural resources, construction, and maintenan...",9510.0,56168.0


,year,l2,SEX,count
0,2003,"Management, professional, and related occupations",Females,48174.0
1,2003,"Natural resources, construction, and maintenan...",Females,9706.0
2,2003,Sales and office occupations,Females,43911.0
3,2003,Service occupations,Females,24020.0
4,2004,"Management, professional, and related occupations",Females,48596.0
...,...,...,...,...
99,2014,Service occupations,Males,21564.0
100,2015,"Management, professional, and related occupations",Males,56006.0
101,2015,"Natural resources, construction, and maintenan...",Males,53142.0
102,2015,Sales and office occupations,Males,25254.0


In [261]:
Mng_df = df_melted[df_melted['l2'] == 'Management, professional, and related occupations'].reset_index()
Nat_df = df_melted[df_melted['l2'] == 'Natural resources, construction, and maintenance occupations'].reset_index()
Sal_df = df_melted[df_melted['l2'] == 'Sales and office occupations'].reset_index()
Ser_df = df_melted[df_melted['l2'] == 'Service occupations'].reset_index()

lst = [Mng_df, Nat_df, Sal_df, Ser_df]
Mng_df

,index,year,l2,SEX,count
0,0,2003,"Management, professional, and related occupations",Females,48174.0
1,4,2004,"Management, professional, and related occupations",Females,48596.0
2,8,2005,"Management, professional, and related occupations",Females,49586.0
3,12,2006,"Management, professional, and related occupations",Females,50784.0
4,16,2007,"Management, professional, and related occupations",Females,52197.0
5,20,2008,"Management, professional, and related occupations",Females,53449.0
6,24,2009,"Management, professional, and related occupations",Females,53493.0
7,28,2010,"Management, professional, and related occupations",Females,53175.0
8,32,2011,"Management, professional, and related occupations",Females,53839.0
9,36,2012,"Management, professional, and related occupations",Females,55468.0


In [262]:
for i in lst:
        area = alt.Chart(i).mark_area(opacity = 0.4).encode(
                x="year",
                y=alt.Y('count', stack = False),
                color="SEX",
                ).properties(title={"text": i['l2'][0]}
                )
        display(area)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)